In [2]:
import pandas as pd
pd.set_option("display.max_columns", 120)

LISTINGS = "../data/processed/listings_clean.csv"
CALENDAR = "../data/raw/calendar.csv.gz"

listings = pd.read_csv(LISTINGS)
listings.head(2), listings.shape


(     id  latitude  longitude   neighbourhood_cleansed        room_type  \
 0  3176  52.53471   13.41810  Prenzlauer Berg Südwest  Entire home/apt   
 1  9991  52.53269   13.41805  Prenzlauer Berg Südwest  Entire home/apt   
 
         property_type  accommodates  bedrooms  bathrooms_num  minimum_nights  \
 0  Entire rental unit             2       1.0            1.0              63   
 1  Entire rental unit             7       4.0            2.5               6   
 
    price_eur  review_scores_rating  number_of_reviews  
 0      105.0                  4.63                149  
 1      135.0                  5.00                  7  ,
 (9183, 13))

In [3]:
cal = pd.read_csv(CALENDAR, low_memory=False, parse_dates=["date"])
cal.head(2), cal.shape, cal.columns.tolist()


(   listing_id       date available  price  adjusted_price  minimum_nights  \
 0     1358910 2025-09-24         f    NaN             NaN               3   
 1     1358910 2025-09-25         f    NaN             NaN               3   
 
    maximum_nights  
 0             365  
 1             365  ,
 (5210011, 7),
 ['listing_id',
  'date',
  'available',
  'price',
  'adjusted_price',
  'minimum_nights',
  'maximum_nights'])

In [4]:
cal["date"] = pd.to_datetime(cal["date"], errors="coerce")
cal["available"] = cal["available"].map({"t": True, "f": False})




In [5]:
cutoff = cal["date"].max() - pd.Timedelta(days=90)
recent = cal[cal["date"] >= cutoff]
recent["available"].isna().mean(), recent["date"].min(), recent["date"].max()







(np.float64(0.0),
 Timestamp('2026-06-25 00:00:00'),
 Timestamp('2026-09-23 00:00:00'))

In [6]:
# Hinweis: Calendar hat die Spalte 'listing_id'
unavail = (recent.groupby("listing_id")["available"]
                 .apply(lambda s: 1 - s.mean())
                 .rename("unavailable_rate_90d"))
unavail.head()


listing_id
3176     0.000000
9991     0.186813
14325    0.000000
17904    0.329670
20858    1.000000
Name: unavailable_rate_90d, dtype: float64

In [7]:
# Listings-Tabelle hat 'id' als Schlüssel
merged = listings.merge(unavail, left_on="id", right_index=True, how="left")
merged["unavailable_rate_90d"] = merged["unavailable_rate_90d"].fillna(0.0)  # falls kein Calendar-Verlauf
merged.shape, merged[["price_eur","unavailable_rate_90d"]].head(3)


((9183, 14),
    price_eur  unavailable_rate_90d
 0      105.0              0.000000
 1      135.0              0.186813
 2       75.0              0.000000)

In [37]:
merged["unavailable_rate_90d"].describe()
merged[["price_eur","unavailable_rate_90d"]].corr()


,price_eur,unavailable_rate_90d
price_eur,1.000000,-0.051409
unavailable_rate_90d,-0.051409,1.000000


In [38]:
OUT = "../data/processed/merged.csv"
merged.to_csv(OUT, index=False)
OUT


'../data/processed/merged.csv'

- Demand-Proxy = Anteil der Tage in den letzten 90 Tagen, an denen das Listing NICHT verfügbar war.
- Caveat: "unavailable" enthält sowohl gebuchte als auch host-blockierte Tage.
